# Hydraulijärjestelmän mallin tarkkuustesti

Tässä notebookissa testataan hydraulijärjestelmän tilan luokitteluun käytetyn mallin tarkkuutta. Notebook sisältää testausosion testHydraulicDataAccuracy.py-tiedostosta.

## Kirjastojen lataus

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Määritä tiedostojen nimet
MODEL_FILENAME = "SVM.pkl"
SCALER_FILENAME = "hydraulicScaler.joblib"

## Hydraulisen datan lataus

Ladataan alkuperäinen generoitu hydraulicData.csv-tiedosto, joka sisältää hydraulijärjestelmän tilan luokitteluun käytettävät ominaisuudet ja luokat.

In [2]:
def loadHydraulicData():
    """
    Lataa hydraulicData.csv-tiedosto
    """
    try:
        # Määritä tiedostojen polut
        currentDir = os.path.dirname(os.path.abspath("__file__"))
        dataDir = os.path.join(os.path.dirname(currentDir), 'Data')
        
        # Lataa hydraulicData.csv
        hydraulicDataPath = os.path.join(dataDir, 'hydraulicData.csv')
        
        if not os.path.exists(hydraulicDataPath):
            print(f"Virhe: Tiedostoa {hydraulicDataPath} ei löydy")
            sys.exit(1)
        
        hydraulicData = pd.read_csv(hydraulicDataPath)
        
        print(f"Ladattu hydraulicData: {hydraulicDataPath}")
        print(f"HydraulicDatan koko: {hydraulicData.shape}")
        print(f"HydraulicDatan sarakkeet: {hydraulicData.columns.tolist()}")
        
        # Erota syöte- ja kohdemuuttujat
        xHydraulic = hydraulicData[['pumpControl', 'pressure']]
        yHydraulic = hydraulicData['state']
        
        # Muunna luokkamuuttujat numeerisiksi
        labelEncoder = LabelEncoder()
        yHydraulicEncoded = labelEncoder.fit_transform(yHydraulic)
        
        # Tallenna luokkien nimet
        classNames = labelEncoder.classes_
        
        print("\nEsimerkkejä hydraulicDatasta:")
        print(hydraulicData[['pumpControl', 'pressure', 'state']].head())
        
        print("\nTilojen jakauma hydraulicDatassa:")
        print(hydraulicData['state'].value_counts())
        
        print("\nHydraulicDatan tilastot:")
        print(hydraulicData[['pumpControl', 'pressure']].describe())
        
        # Tarkista arvoalueet
        print("\nHydraulicDatan arvoalueet:")
        print(f"pumpControl: [{hydraulicData['pumpControl'].min()}, {hydraulicData['pumpControl'].max()}]")
        print(f"pressure: [{hydraulicData['pressure'].min()}, {hydraulicData['pressure'].max()}]")
        
        print("\nLuokkien koodaus:")
        for i, className in enumerate(classNames):
            print(f"{className} -> {i}")
        
        return xHydraulic, yHydraulicEncoded, classNames
    except Exception as e:
        print(f"Virhe hydraulicDatan lataamisessa: {e}")
        sys.exit(1)

# Ladataan hydraulinen data
xHydraulic, yHydraulic, classNames = loadHydraulicData()

Ladattu hydraulicData: d:\Rakentelu\Projektit\10_LED_ohjain_2313_baremetal_ML\HydraulicControlML\Data\hydraulicData.csv
HydraulicDatan koko: (10000, 3)
HydraulicDatan sarakkeet: ['pumpControl', 'pressure', 'state']

Esimerkkejä hydraulicDatasta:
   pumpControl  pressure           state
0           71       179  normal_digging
1           71       190  normal_digging
2           69       178  normal_digging
3           73       185  normal_digging
4           69       183  normal_digging

Tilojen jakauma hydraulicDatassa:
state
normal_digging    3685
overload          3000
normal_full       1205
normal_empty      1110
hose_break        1000
Name: count, dtype: int64

HydraulicDatan tilastot:
        pumpControl      pressure
count  10000.000000  10000.000000
mean      71.314900    193.034800
std       22.347588    113.335096
min       18.000000      1.000000
25%       59.000000    140.000000
50%       72.000000    176.000000
75%       93.000000    319.000000
max      101.000000    432.0

## Mallin ja skaalerin lataus

Ladataan koulutettu SVM-malli ja skaalauskomponentti.

In [3]:
def loadModelAndScaler():
    """
    Lataa malli ja skaaleri
    """
    try:
        # Määritä tiedostojen polut
        currentDir = os.path.dirname(os.path.abspath("__file__"))
        modelDir = os.path.join(os.path.dirname(currentDir), 'Models')
        
        # Lataa malli ja skaaleri
        modelPath = os.path.join(modelDir, MODEL_FILENAME)
        scalerPath = os.path.join(modelDir, SCALER_FILENAME)
        
        if not os.path.exists(modelPath) or not os.path.exists(scalerPath):
            print(f"Virhe: Tiedostoja {modelPath} tai {scalerPath} ei löydy")
            sys.exit(1)
        
        # Lataa malli ja skaaleri
        import pickle
        import joblib
        
        with open(modelPath, 'rb') as f:
            model = pickle.load(f)
        
        scaler = joblib.load(scalerPath)
        
        return model, scaler
    except Exception as e:
        print(f"Virhe mallin ja skaalerin lataamisessa: {e}")
        sys.exit(1)

# Ladataan malli ja skaaleri
model, scaler = loadModelAndScaler()

print(f"Käytössä malli: {MODEL_FILENAME}")
print(f"Käytössä skaaleri: {SCALER_FILENAME}")

Käytössä malli: SVM.pkl
Käytössä skaaleri: hydraulicScaler.joblib


## Datan skaalaus ja ennusteiden tekeminen

Skaalataan syötearvot välille [0, 1] ja tehdään ennusteet.

In [5]:
# Skaalaa arvot manuaalisesti välille [0, 1]
print("\nSkaalataan arvot...")

# Määritä skaalauksen rajat
pumpControlMin = 18
pumpControlMax = 101
pressureMin = 1
pressureMax = 432

# Skaalaa arvot
xHydraulicScaled = xHydraulic.copy()
xHydraulicScaled['pumpControl'] = xHydraulic['pumpControl'].apply(
    lambda x: max(0, min(1, (x - pumpControlMin) / (pumpControlMax - pumpControlMin)))
)
xHydraulicScaled['pressure'] = xHydraulic['pressure'].apply(
    lambda x: max(0, min(1, (x - pressureMin) / (pressureMax - pressureMin)))
)

print("Skaalatut arvot (ensimmäiset 5 riviä):")
print(xHydraulicScaled.head())

# Tee ennusteet
predictions = model.predict(xHydraulicScaled)

# Laske tarkkuus
accuracy = accuracy_score(yHydraulic, predictions)

print(f"\nMallin tarkkuus: {accuracy:.4f} ({accuracy*100:.2f}%)")

# Näytä ennusteiden jakauma
print("\nEnnusteiden jakauma:")
print(pd.Series(predictions).value_counts())

# Näytä luokitteluraportti
classReport = classification_report(yHydraulic, predictions, target_names=classNames)
print("\nLuokitteluraportti:")
print(classReport)

# Laske sekaannusmatriisi
allLabels = np.arange(len(classNames))
cm = confusion_matrix(yHydraulic, predictions, labels=allLabels)

# Valmistele data visualisointia varten
hydraulicDataWithPredictions = pd.DataFrame({
    'pumpControl': xHydraulic['pumpControl'],
    'pumpControlScaled': xHydraulicScaled['pumpControl'],
    'pressure': xHydraulic['pressure'],
    'pressureScaled': xHydraulicScaled['pressure'],
    'actual': [classNames[i] for i in yHydraulic],
    'predicted': [classNames[i] for i in predictions]
})

# Tunnista virheelliset luokittelut
errors = hydraulicDataWithPredictions[hydraulicDataWithPredictions['actual'] != hydraulicDataWithPredictions['predicted']]
print(f"Virheellisiä luokitteluja: {len(errors)} / {len(hydraulicDataWithPredictions)} ({len(errors)/len(hydraulicDataWithPredictions)*100:.2f}%)")

# Käytä olemassa olevaa visualisointikansiota
def getVisualizationDirectory():
    """
    Palauttaa olemassa olevan Visualization-kansion polun
    """
    currentDir = os.path.dirname(os.path.abspath("__file__"))
    projectRoot = os.path.dirname(currentDir)
    visualizationDir = os.path.join(projectRoot, 'Visualization')
    
    if not os.path.exists(visualizationDir):
        os.makedirs(visualizationDir)
        print(f"Luotu Visualization-kansio: {visualizationDir}")
    
    return visualizationDir

# Käytä olemassa olevaa visualisointikansiota
visualizationDir = getVisualizationDirectory()
print(f"Käytetään visualisointikansiota: {visualizationDir}")


Skaalataan arvot...
Skaalatut arvot (ensimmäiset 5 riviä):
   pumpControl  pressure
0     0.638554  0.412993
1     0.638554  0.438515
2     0.614458  0.410673
3     0.662651  0.426914
4     0.614458  0.422274

Mallin tarkkuus: 0.9902 (99.02%)

Ennusteiden jakauma:
1    3705
4    2989
3    1209
2    1115
0     982
Name: count, dtype: int64

Luokitteluraportti:
                precision    recall  f1-score   support

    hose_break       1.00      0.98      0.99      1000
normal_digging       0.99      0.99      0.99      3685
  normal_empty       1.00      1.00      1.00      1110
   normal_full       0.96      0.96      0.96      1205
      overload       1.00      1.00      1.00      3000

      accuracy                           0.99     10000
     macro avg       0.99      0.99      0.99     10000
  weighted avg       0.99      0.99      0.99     10000

Virheellisiä luokitteluja: 98 / 10000 (0.98%)
Käytetään visualisointikansiota: d:\Rakentelu\Projektit\10_LED_ohjain_2313_baremetal

## Sekaannusmatriisin laskeminen

Lasketaan ja näytetään sekaannusmatriisi.

In [6]:
# Näytä sekaannusmatriisi
print("\nSekaannusmatriisi:")
print("Todellinen luokka (rivit) vs. Ennustettu luokka (sarakkeet):")
print("=" * 80)

# Varmista, että kaikki luokat ovat edustettuina sekaannusmatriisissa
allLabels = np.arange(len(classNames))

cm = confusion_matrix(yHydraulic, predictions, labels=allLabels)
cmDf = pd.DataFrame(cm, index=classNames, columns=classNames)
print(cmDf)

# Näytä virheellisesti luokitellut näytteet
hydraulicDataWithPredictions = pd.DataFrame({
    'pumpControl': xHydraulic['pumpControl'],
    'pumpControlScaled': xHydraulicScaled['pumpControl'],
    'pressure': xHydraulic['pressure'],
    'pressureScaled': xHydraulicScaled['pressure'],
    'actual': [classNames[i] for i in yHydraulic],
    'predicted': [classNames[i] for i in predictions]
})

errors = hydraulicDataWithPredictions[hydraulicDataWithPredictions['actual'] != hydraulicDataWithPredictions['predicted']]

print("\nVirheellisesti luokitellut näytteet:")
if len(errors) > 0:
    print(errors.head(20))
    print(f"\nYhteensä {len(errors)} virheellistä luokittelua {len(hydraulicDataWithPredictions)} näytteestä.")
    print(f"Virheprosentti: {len(errors)/len(hydraulicDataWithPredictions)*100:.2f}%")
else:
    print("Ei virheellisiä luokitteluja!")


Sekaannusmatriisi:
Todellinen luokka (rivit) vs. Ennustettu luokka (sarakkeet):
                hose_break  normal_digging  normal_empty  normal_full  \
hose_break             982               1             5           12   
normal_digging           0            3660             0           25   
normal_empty             0               0          1110            0   
normal_full              0              44             0         1161   
overload                 0               0             0           11   

                overload  
hose_break             0  
normal_digging         0  
normal_empty           0  
normal_full            0  
overload            2989  

Virheellisesti luokitellut näytteet:
      pumpControl  pumpControlScaled  pressure  pressureScaled  \
239            76           0.698795       188        0.433875   
394            74           0.674699       186        0.429234   
582            72           0.650602       197        0.454756   
798            7

## Tekstiraportin tallennus

Tallennetaan tekstiraportti tuloksista.

In [7]:
def saveTextReport(hydraulicData, accuracy, classReport, cmDf, visualizationDir):
    """
    Tallentaa tekstiraportin tuloksista
    """
    reportPath = os.path.join(visualizationDir, 'accuracy_report.txt')
    
    with open(reportPath, 'w') as f:
        f.write("HYDRAULIIKKAJÄRJESTELMÄN MALLIN TARKKUUSRAPORTTI\n")
        f.write("=" * 50 + "\n\n")
        
        f.write(f"Päivämäärä: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        f.write(f"Käytetty malli: {MODEL_FILENAME}\n")
        f.write(f"Käytetty skaaleri: {SCALER_FILENAME}\n\n")
        
        f.write("YHTEENVETO\n")
        f.write("-" * 30 + "\n")
        f.write(f"Näytteiden määrä: {len(hydraulicData)}\n")
        f.write(f"Mallin tarkkuus: {accuracy:.4f} ({accuracy*100:.2f}%)\n")
        
        errors = hydraulicData[hydraulicData['actual'] != hydraulicData['predicted']]
        f.write(f"Virheellisiä luokitteluja: {len(errors)} / {len(hydraulicData)} ({len(errors)/len(hydraulicData)*100:.2f}%)\n\n")
        
        f.write("LUOKKIEN JAKAUMAT\n")
        f.write("-" * 30 + "\n")
        f.write("Todelliset luokat:\n")
        actualCounts = hydraulicData['actual'].value_counts()
        for className, count in actualCounts.items():
            f.write(f"  {className}: {count} ({count/len(hydraulicData)*100:.2f}%)\n")
        
        f.write("\nEnnustetut luokat:\n")
        predictedCounts = hydraulicData['predicted'].value_counts()
        for className, count in predictedCounts.items():
            f.write(f"  {className}: {count} ({count/len(hydraulicData)*100:.2f}%)\n\n")
        
        f.write("LUOKITTELURAPORTTI\n")
        f.write("-" * 30 + "\n")
        f.write(classReport + "\n\n")
        
        f.write("SEKAANNUSMATRIISI\n")
        f.write("-" * 30 + "\n")
        f.write("Todellinen luokka (rivit) vs. Ennustettu luokka (sarakkeet):\n\n")
        f.write(cmDf.to_string() + "\n\n")
        
        f.write("VIRHEANALYYSI\n")
        f.write("-" * 30 + "\n")
        
        if len(errors) > 0:
            # Virheet luokittain
            errorsByClass = {}
            for actualClass in hydraulicData['actual'].unique():
                errorsByClass[actualClass] = {}
                for predictedClass in hydraulicData['predicted'].unique():
                    if actualClass != predictedClass:
                        count = len(errors[(errors['actual'] == actualClass) & (errors['predicted'] == predictedClass)])
                        if count > 0:
                            errorsByClass[actualClass][predictedClass] = count
            
            f.write("Virheet luokittain:\n")
            for actualClass, predictions in errorsByClass.items():
                if predictions:
                    f.write(f"  {actualClass} luokiteltu väärin:\n")
                    for predictedClass, count in predictions.items():
                        f.write(f"    {predictedClass}: {count} kpl\n")
            
            f.write("\nEsimerkkejä virheellisistä luokitteluista:\n")
            f.write(errors[['pumpControl', 'pumpControlScaled', 'pressure', 'pressureScaled', 'actual', 'predicted']].head(10).to_string() + "\n")
        else:
            f.write("Ei virheellisiä luokitteluja!\n")
    
    print(f"Tekstiraportti tallennettu: {reportPath}")

# Tallennetaan tekstiraportti
saveTextReport(hydraulicDataWithPredictions, accuracy, classReport, cmDf, visualizationDir)

Tekstiraportti tallennettu: d:\Rakentelu\Projektit\10_LED_ohjain_2313_baremetal_ML\HydraulicControlML\Visualization\accuracy_report.txt
